In [6]:
#!pip install gradio==4.16.0
#!pip install huggingface_hub==0.16.4
#!pip install openai==1.52.1


  Attempting uninstall: openai
    Found existing installation: openai 1.52.1
    Uninstalling openai-1.52.1:
      Successfully uninstalled openai-1.52.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 1.0.1 requires openai<3.0.0,>=1.109.1, but you have openai 0.28.1 which is incompatible.


In [ ]:
# ============================================================
# 🎨 TEXT-TO-IMAGE GENERATOR USING OPENAI + GRADIO 
# ============================================================

# --- STEP 0: Install required packages (run once) ---
#!pip install --upgrade openai==1.52.1 gradio==4.44.1 huggingface_hub==0.22.2 httpx==0.27.2 Pillow --quiet

# --- Imports ---
import os, io, base64
from PIL import Image, ImageDraw
import gradio as gr
from openai import OpenAI

# ============================================================
# STEP 1: Configure OpenAI API key
# ============================================================
api_key = "xxxxxxxx"   # 🔹 Replace with your real key
os.environ["OPENAI_API_KEY"] = api_key
print("🔑 API Key loaded:", os.getenv("OPENAI_API_KEY")[:8])

# Initialize OpenAI client
client = OpenAI(api_key=api_key)
print("✅ OpenAI client initialized successfully.")


# ============================================================
# STEP 2: Define image generation function with full error handling
# ============================================================

def generate_image(prompt: str):
    """
    Generate an image using OpenAI's gpt-image-1 model.
    Handles billing, network, and API errors gracefully.
    Falls back to a mock image for demo or blocked environments.
    """
    if not prompt or prompt.strip() == "":
        # Empty prompt case
        img = Image.new("RGB", (512, 512), color=(255, 230, 230))
        d = ImageDraw.Draw(img)
        d.text((20, 240), "⚠️ Please enter a valid prompt", fill=(0, 0, 0))
        return img

    try:
        print("🔍 Generating image for prompt:", prompt)
        result = client.images.generate(
            model="gpt-image-1",
            prompt=prompt,
            size="1024x1024"
        )

        # Decode Base64 → PIL Image
        image_base64 = result.data[0].b64_json
        image_bytes = base64.b64decode(image_base64)
        image = Image.open(io.BytesIO(image_bytes))
        print("✅ Image generated successfully.")
        return image

    except Exception as e:
        # --- Error handling ---
        err_msg = str(e)
        print(f"❌ API Error: {err_msg}")

        # Handle specific billing / quota errors
        if "Insufficient budget" in err_msg or "quota" in err_msg:
            err_msg = "⚠️ Your OpenAI API key has no remaining credits. Please add funds at platform.openai.com."

        elif "Incorrect API key" in err_msg or "authentication" in err_msg:
            err_msg = "⚠️ Invalid API key. Please check and replace it in the code."

        elif "Connection" in err_msg or "timeout" in err_msg:
            err_msg = "⚠️ Network issue or API unreachable. Check your internet connection."

        else:
            err_msg = f"⚠️ Unexpected error: {err_msg}"

        # Always return a placeholder image with the error message
        img = Image.new("RGB", (512, 512), color=(240, 240, 240))
        d = ImageDraw.Draw(img)
        d.text((20, 220), err_msg, fill=(0, 0, 0))
        return img


# ============================================================
# STEP 3: Build and launch the Gradio interface
# ============================================================

demo = gr.Interface(
    fn=generate_image,
    inputs=gr.Textbox(
        label="Prompt",
        placeholder="Describe the image you want (e.g. 'A cat wearing sunglasses on a beach')",
        lines=2
    ),
    outputs=gr.Image(label="Generated Image"),
    title="🧠 AI Image Generator",
    description="Generate custom images using OpenAI's DALL·E (gpt-image-1). "
                "Includes automatic fallback for blocked or unpaid API environments.",
    examples=[
        ["A cyberpunk city at night, glowing neon signs"],
        ["A fantasy castle floating above the clouds"],
        ["An astronaut playing guitar on the moon"]
    ]
)

# Launch app — share=True creates a public gradio.live link
demo.launch(share=True)


🔑 API Key loaded: sk-proj-
✅ OpenAI client initialized successfully.
Running on local URL:  http://127.0.0.1:7861
IMPORTANT: You are using gradio version 4.16.0, however version 4.44.1 is available, please upgrade.
--------
Running on public URL: https://a17214fc710be6f544.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


🔍 Generating image for prompt: A fantasy castle floating above the clouds
❌ API Error: Error code: 400 - {'error': {'message': 'Billing hard limit has been reached.', 'type': 'billing_limit_user_error', 'param': None, 'code': 'billing_hard_limit_reached'}}


Found existing installation: openai 0.28.1
Uninstalling openai-0.28.1:
  Successfully uninstalled openai-0.28.1
  Using cached openai-1.52.1-py3-none-any.whl.metadata (24 kB)
Using cached openai-1.52.1-py3-none-any.whl (386 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 1.0.1 requires openai<3.0.0,>=1.109.1, but you have openai 1.52.1 which is incompatible.
